In [ ]:
import cfr
import pandas as pd
import numpy as np
import pyleoclim as pyleo
import matplotlib.pyplot as plt

## Combining metadata and data pickle files 

### Loading metadata and data csv file and filtering only PAGES2kv2 data

In [ ]:
# loading metadata

df_meta = pd.read_csv('./proxydb_meta.csv')
df_p2k_meta = df_meta[df_meta['Proxy ID'].str.contains('PAGES2kv2')]
df_p2k_meta.set_index('Proxy ID', inplace=True)

In [ ]:
archive_data = df_p2k_meta[['Archive type', 'Proxy measurement', 'Lat (N)', 'Lon (E)', 'Elev']]
archive_data['Proxy ID'] = archive_data.index  # recover it as a column


In [ ]:
# loading data (time and value)

df = pd.read_csv('./proxydb_data.csv')
col = df.columns

clist = []
for c in col:
    if ('PAGES2kv2' in c) or ('Year C.E.' in c):
        clist.append(c)

p2k = df[clist]
p2k = p2k.reset_index(drop=True)

### function for 'pid' column to follow CFR 

In [ ]:
time_col = p2k.columns[0]

pids = []
times = []
values = []

for pid in p2k.columns[1:]:  # Skip the time column
        # Get the time series for this proxy
        proxy_data = p2k[[time_col, pid]].dropna()
        
        if len(proxy_data) > 0:  # Only process if we have data
            pids.append(pid)
            times.append(proxy_data[time_col].tolist())
            values.append(proxy_data[pid].tolist())

In [ ]:
def shorten_pid(pids): 
    # Split by colon first to handle proxy measurements with underscores
    parts_by_colon = pids.split(':')[0]  # Take everything before the colon
    parts = parts_by_colon.split('_')
    
    # Find the part that contains the number
    number = None
    for part in parts:
        if part.isdigit():
            number = part
            break
        elif 'NAm_' in part:
            number = part.split('_')[1]
            break
        elif 'Asi_' in part:
            number = part.split('_')[1]
            break
        elif 'Arc_' in part:
            number = part.split('_')[1]
            break
        elif 'Ant_' in part:
            number = part.split('_')[1]
            break
        elif 'Ocn_' in part:
            number = part.split('_')[1]
            break
        elif 'Aus_' in part:
            number = part.split('_')[1]
            break
    
    # Extract region from the second part
    region = parts[1].split('-')[0]
    
    # Map the region
    region_mapping = {
        'Asia': 'Asi',
        'Ocean2kHR': 'Ocn',
        'Africa': 'Afr',
        'Afr': 'Afr',
        'NAm': 'NAm',
        'Arc': 'Arc',
        'Ant': 'Ant',
        'Aus': 'Aus',
        'SAm': 'SAm'
    }
    region = region_mapping.get(region, region)
    
    if number is None:
        # If we still haven't found a number, look for it specifically
        for part in parts:
            if any(char.isdigit() for char in part):
                number = ''.join(char for char in part if char.isdigit())
                break
    
    return f"{region}_{number}"

In [ ]:
short_pids = []

for p in pids:
    lil = shorten_pid(p)
    short_pids.append(lil)

df_p2k_meta_short = df_p2k_meta.rename(index=shorten_pid)
archive_data_new_idx = archive_data.rename(index=shorten_pid)


data_df = pd.DataFrame({
        'pid': short_pids,
        'time': times,
        'value': values
    })

In [ ]:
archive_data_new_idx = archive_data_new_idx.reset_index(names=['pid'])

new_pdb = pd.merge(
        archive_data_new_idx,
        data_df,
        on='pid',
        how='inner'
)

In [ ]:
def create_ptype(row):

    archive_mapping = {
        'Tree Rings': 'tree',
        'Corals and Sclerosponges': 'coral',
        'Lake Cores': 'lake',
        'Ice Cores': 'ice',
        'Bivalve': 'bivalve',
        'Speleothems': 'speleothem',
        'Marine Cores': 'marine'
    }

    measure_mapping = {
        'Sr_Ca': 'SrCa',
        'trsgi': 'TRW',
        'calcification': 'calc',
        'd18O': 'd18O',
        'dD': 'dD',
        'MXD': 'MXD',
        'density': 'MXD',
        'composite': 'calc',
        'massacum': 'accumulation',
        'thickness': 'varve_thickness',
        'melt': 'melt',
        'RABD660_670': 'reflectance',
        'X_radiograph_dark_layer': 'varve_property'
    }
    
    # Get the shortened archive type
    archive = archive_mapping.get(row['Archive type'], row['Archive type'].lower())
    proxy = measure_mapping.get(row['Proxy measurement'])
    
    # Combine with proxy measurement
    return f"{archive}.{proxy}"

In [ ]:
new_pdb['ptype'] = new_pdb.apply(create_ptype, axis=1)
new_pdb = new_pdb.drop(['Archive type', 'Proxy measurement'], axis=1)

## making function for sub-annual filtering (from LMR)

In [ ]:

def is_subannual(time_array):
    if len(time_array) < 2:
        return False
    dt = np.median(np.diff(time_array))
    return dt < 0.95  # annual resolution threshold (from LMR)

def compute_annual_means_for_row(row, valid_frac=0.5):
    pid = row['pid']

    # Manually skip annualizing for this record
    if pid == 'Ocn_148':
        return row['time'], row['value'], False 
    
    time = np.array(row['time'], dtype=float)
    value = np.array(row['value'], dtype=float)

    # Drop NaNs
    mask = ~np.isnan(time) & ~np.isnan(value)
    time = time[mask]
    value = value[mask]

    if len(time) < 2:
        return None, None, False

    subannual = is_subannual(time)

    if subannual:
        # Bin into annual averages
        df = pd.DataFrame({'year': time.astype(int), 'value': value})
        annual = df.groupby('year').mean().dropna()
        
        year_range = annual.index.max() - annual.index.min() + 1
        coverage = len(annual) / year_range if year_range > 0 else 0
        
        if coverage < valid_frac:
            return None, None, subannual
        
        return annual.index.to_list(), annual['value'].to_list(), subannual
    
    else:
        # Assume already annual — check coverage
        years = time.astype(int)
        year_range = years.max() - years.min() + 1
        coverage = len(years) / year_range if year_range > 0 else 0
        
        if coverage < valid_frac:
            return None, None, subannual
        
        return years.tolist(), value.tolist(), subannual


In [ ]:
results = new_pdb.apply(lambda row: compute_annual_means_for_row(row), axis=1)

new_pdb[['annual_time', 'annual_value', 'is_subannual']] = pd.DataFrame(results.tolist(), index=new_pdb.index)
new_pdb = new_pdb[new_pdb['pid'] != 'Afr_012']

# Drop records where annualization failed
new_pdb_up = new_pdb[new_pdb['annual_time'].notnull()]


new_pdb_up = new_pdb_up.drop(columns=['annual_time', 'annual_value', 'is_subannual', 'Proxy ID'])


In [ ]:
new_pdb_up

## For use in cfr 

In [ ]:
new_pdb_up = new_pdb_up.rename(columns={
    'Lat (N)': 'lat',
    'Lon (E)': 'lon',
    'Elev': 'elev'
})

In [ ]:
new_pdb_up

In [ ]:
new_pdb_up.to_json('./prev_data/updated_pages2kv2.json', orient='records')

In [ ]:
job = cfr.ReconJob()
job.load_proxydb(
    path='./prev_data/updated_pages2kv2.json', 
    verbose=True,
    pid_column='pid', 
    lat_column='lat',  
    lon_column='lon',  
    elev_column = 'elev',
    time_column = 'time',
    value_column = 'value',
    ptype_column = 'ptype'

)

fig, ax = job.proxydb.plot(plot_count=True)